In [101]:
from lxml import html
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from shutil import which
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from datetime import datetime
from lxml import etree
import re
import time
from bs4 import BeautifulSoup
import pyodbc
import sqlalchemy as sa
from sqlalchemy.types import NVARCHAR
import urllib
import glob
from selenium.webdriver.common.action_chains import ActionChains
# from requests_auth_aws_sigv4 import AWSSigV4
# import boto3
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
##from selenium_move_cursor.MouseActions import move_to_element_chrome
# import chromedriver_autoinstaller
from pyspark.sql import SparkSession
import pyodbc
import sqlalchemy as sa
from sqlalchemy.orm import Session
import urllib
import pyspark.sql.functions as f


In [84]:
def create_connection():
    params = urllib.parse.quote_plus(
        'Driver={ODBC Driver 17 for SQL Server};'
        'Server=server_name;'
        'Database=database_name;'
        'Trusted_connection=yes;'
    )
    engine = sa.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(params))
    return engine

def write_to_database(df,table_name):
    engine=create_connection()
    df.to_sql(str(table_name),con=engine, if_exists='append',index=False)

In [90]:
def search_items():
        chrome_options=Options()
        chrome_options.add_argument("--incognito")

        driver = webdriver.Chrome(executable_path="C:\chromedriver.exe",options=chrome_options)
        # chromedriver_autoinstaller.install() #use this to autoinstall chromedriver instead of giving explicit path
        driver = webdriver.Chrome()
        result = []  #to store final data

        keywords=['fortune','cadbury'] #keywords to search

        for keyword in keywords:
                for num in range(1,5): #page number range for search
                        url = 'https://www.amazon.in/s?k=' + str(keyword) + '&page=' + str(num)
                        driver.get(url)
                        time.sleep(4)
                        try:
                                driver.find_element_by_xpath('//span[contains(text(),"Search instead for ")]').click() #alternate search click
                        except:
                                pass
                        time.sleep(3)
                        
                        html = driver.page_source
                        element_tree = etree.HTML(html)                
                        products =  element_tree.xpath('//div[@class="s-main-slot s-result-list s-search-results sg-row"]/div')
                        print(len(products))
                        for product in products:
                                base_url = 'https://www.amazon.in'
                                
                                item_name = product.xpath('.//span[@class="a-size-base-plus a-color-base a-text-normal"]/text()')\
                                        or product.xpath('.//span[@class="a-size-medium a-color-base a-text-normal"]/text()')\
                                        or product.xpath('.//div[@class="a-section a-spacing-medium"]/div[2]/div[1]/h2/a/span/text()')
                                item_name = ''.join(item_name)
                                item_name = item_name.lower()

                                if keyword.lower() in item_name.lower() or keyword.lower().replace(' ','') in item_name.lower():
                                        check='True'
                                        print(item_name)
                                        print(check)
                                else:
                                        check='False'
        ##                        print(check)
                                if check=="True":

                                        product_url= product.xpath(".//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-4']/a/@href")\
                                                or product.xpath('.//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]/@href')
                                        main_url = ['https://www.amazon.in']
                                        full_url = main_url+product_url
                                        link = "".join(full_url)

                        
                                        try:
                                                item_code = re.search(r'/[dg]p/([^/]+)', link, flags=re.IGNORECASE).group(1)
                                                if item_code=='slredirect':
                                                        item_code = re.findall('dp%2F(.*?)%2F',link)
                                                        item_code = ''.join(item_code)
                                                else:
                                                        item_code = item_code	

                                        except:
                                                item_code = ''
                                        
                                        dict1 = {}
                                        dict1["input_keyword"]=keyword
                                        dict1["item_code"] = item_code
                                        dict1["product_name"] = item_name
                                        dict1['product_url'] = link
                                        print(dict1)
                                        result.append(dict1)
                        
        df = pd.DataFrame(result)
        df.drop_duplicates(subset ="item_code", inplace = True)
        df.to_excel('item_urls.xlsx', index = False)	
        write_to_database(df,'item_search') #push dataframe generated to Database
        driver.close()
        driver.quit()


In [118]:
def get_item_details(codes):
    chrome_options=Options()
    chrome_path=which("chromedriver")
    driver = webdriver.Chrome(executable_path="C:/chromedriver.exe",options=chrome_options)
    url='https://www.amazon.in/dp/'
    result=[]
    for code in codes:
        dict1={}
        driver.get(code)
        time.sleep(3)
        
        html = driver.page_source
        element_tree = etree.HTML(html)


        invalid= element_tree.xpath("//tbody/tr/td[2]/b/text()")
        if invalid:
            continue

        name=element_tree.xpath("//span[@id='productTitle']/text()")
        name=''.join(name).strip()

        no_of_images = element_tree.xpath('//li[@class="a-spacing-small item imageThumbnail a-declarative"]//img')
        if len(no_of_images)== 0:
            no_of_image='1'
        else:
            no_of_image = len(no_of_images)   

        wayfinder=element_tree.xpath('//div[contains(@id,"wayfinding-breadcrumbs")]//li//text()')
        wayfinder=[x.strip().replace('>','') for x in wayfinder]
        wayfinder='|'.join(wayfinder)
        print(wayfinder)
        
        no_of_videos_count=element_tree.xpath('//span[@id="videoCount"]//text()')
        no_of_videos_count=''.join(no_of_videos_count)

        characteristics=element_tree.xpath('//div[@class="a-row a-spacing-top-base"]//table/tbody/tr//node()[not(self::script or self::noscript or self::style)]/text()')\
                        or element_tree.xpath('//div[@id="detailBullets_feature_div"]//ul/li/span//node()[not(self::script or self::noscript or self::style)]/text()')
    ##    
        for chara in characteristics:
            chara=chara.strip().replace('\n','')
        characteristic='|'.join(characteristics)
            

        brand= element_tree.xpath("//span[contains(text(),'Brand')]/following::td[1]/span/text()") \
            or element_tree.xpath("//span[@class='a-list-item']/span[starts-with(text(),'Brand')]/following::span[1]/text()")
        brand=''.join(brand).strip()

        mrp1 = element_tree.xpath("//span[@class='priceBlockStrikePriceString a-text-strike']/text()")\
            or element_tree.xpath('//td[contains(text(),"M.R.P.")]/following::span[1]/span[1]/text()')\
            or element_tree.xpath('.//span[contains(text(),"M.R.P.")]/span/span[1]/text()')
        mrp=''.join(mrp1).strip()

        sp = element_tree.xpath('.//span[@class="a-price aok-align-center reinventPricePriceToPayMargin priceToPay"]/span[1]/text()')\
            or element_tree.xpath('(.//tr/*[contains(text(),"Price:")]/following::span[1]/span[1])[1]/text()')\
            or element_tree.xpath('.//div[@id="corePriceDisplay_desktop_feature_div"]/div[1]/span[1]/span/text()')\
            or element_tree.xpath('.//span[@class="a-price aok-align-center reinventPricePriceToPayPadding priceToPay"]/span[1]/text()')
        price = ''.join(sp).strip()

        rating1 = element_tree.xpath("(//span[@class='a-size-medium a-color-base'])/text()")
        rating = ''.join(rating1).strip()
        
        offers = element_tree.xpath('//tr/*[contains(text(),"You Save")]/following::td[1]/span/text()')\
                or element_tree.xpath('//span[contains(@class,"savings")]/text()')
        offer = ''.join(offers).strip()

        status =  element_tree.xpath('.//div[@id="almAvailability_feature_div"]/div/span/text()')\
                or element_tree.xpath('.//div[@id="almBuyBox_feature_div"]//span[@class="a-size-base-plus a-text-bold"]/text()')\
                or element_tree.xpath('.//div[@id="unqualifiedBuyBox"]//span[@id="buybox-see-all-buying-choices"]/span/a/text()') \
                or element_tree.xpath('.//div[@id="availability"]/span/text()')\
                or element_tree.xpath("//span[@class='a-button a-spacing-small a-button-primary a-button-icon']/span/span/text()")\
                or element_tree.xpath("//span[@class='a-button a-spacing-base a-button-base']/span/a/text()")\
                or element_tree.xpath("//div[@id='outOfStock']/div/div/span/text()")\
                or element_tree.xpath("//div[@id='ddmDeliveryMessage']/span/text()")\
                or element_tree.xpath('//div[@id="almBuyBox_feature_div"]//span[@class="a-size-medium a-color-state"]/text()')
        status = ''.join(status).strip()

        dict1["product_name"]=name
        dict1["code"]=code
        dict1["brand"]=brand
        dict1["mrp"]=mrp
        dict1["final_price"] = price
        dict1["category"]=wayfinder
        dict1["rating"] = rating
        dict1["offers"] = offer
        dict1["produuct_status"] = status
        dict1["image_count"] = no_of_image
        dict1["prod_url"] = code
        dict1["video_count"]=no_of_videos_count

        result.append(dict1)
        print(dict1)
        
    df = pd.DataFrame(result)
    df.to_excel('product_info.xlsx',index=False)
    write_to_database(df,'product_info') #push dataframe generated to Database
    
    driver.close()
    driver.quit()

In [92]:
search_items()

C:\Users\haraa\AppData\Local\Temp\ipykernel_1792\2993422297.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:\chromedriver.exe",options=chrome_options)


67
fortune oil, 1 l pouch soyabean health
True
{'input_keyword': 'fortune', 'item_code': 'B00TX50T4K', 'product_name': 'fortune oil, 1 l pouch soyabean health', 'product_url': 'https://www.amazon.in/Fortune-Soyabean-Oil-1L-Pouch/dp/B00TX50T4K/ref=sr_1_2_f3_0o_fs?keywords=fortune&qid=1680446946&sr=8-2'}
fortune oil, 1 l pouch premium kachi ghani pure mustard
True
{'input_keyword': 'fortune', 'item_code': 'B06XH9QPX2', 'product_name': 'fortune oil, 1 l pouch premium kachi ghani pure mustard', 'product_url': 'https://www.amazon.in/Fortune-Kachi-Ghani-Pure-Mustard/dp/B06XH9QPX2/ref=sr_1_3?keywords=fortune&qid=1680446946&sr=8-3'}
fortune sunlite refined sunflower oil, 1l
True
{'input_keyword': 'fortune', 'item_code': 'B00NYZTGEO', 'product_name': 'fortune sunlite refined sunflower oil, 1l', 'product_url': 'https://www.amazon.in/Fortune-Sunlite-Refined-Sunflower-Oil/dp/B00NYZTGEO/ref=sr_1_4?keywords=fortune&qid=1680446946&sr=8-4'}
fortune premium kachi ghani pure mustard oil, 1litre pet bott

In [93]:
############ CREATE SPARKSESSION TO CLEAN OUTPUT OF ITEM_SEARCH READ PRODUCT_CODE WHICH WILL BE USED BY GET_ITEM_DETAILS
try:
    if spark:
        print('found session, closing')
        spark.stop()
    spark= SparkSession.builder\
        .appName('demospark')\
        .getOrCreate()
except:
    spark= SparkSession.builder\
            .appName('demospark')\
            .getOrCreate()
print(spark)

found session, closing


In [94]:
from pyspark.sql.types import StringType, StructType,StructField
search_schema=StructType([
    StructField('input_keyword',StringType(),True),
    StructField('item_code',StringType(),True),
    StructField('product_name',StringType(),True),
    StructField('product_url',StringType(),True),
])

tdf=[]
engine=create_connection()
######## TO READ EACH ROW AND PARALLELIZE #######################################
# with engine.connect() as conn:
#     result=conn.execute(sa.text('select * from item_search'))
#     for row in result:
#         # print(row)
#         tdf.append(row)
# tdf=spark.sparkContext.parallelize(tdf)
# tdf.collect()
# df=tdf.toDF()
# df.printSchema()
# pdf=spark.createDataFrame(data=tdf,schema=search_schema)
##################################################################################
#TO READ TABLE 
pdf=spark.createDataFrame(pd.read_sql_table('item_search',engine.connect()))
pdf.printSchema()
pdf.show(truncate=False)

C:\Users\haraa\AppData\Roaming\Python\Python310\site-packages\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
C:\Users\haraa\AppData\Roaming\Python\Python310\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


root
 |-- input_keyword: string (nullable = true)
 |-- item_code: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_url: string (nullable = true)

+-------------+----------+----------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------+
|input_keyword|item_code |product_name                                                                      |product_url                                                                                                                            |
+-------------+----------+----------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------+
|fortune      |B09PDZKPHB|fortune indori poha, super soft, 500g                    

In [95]:
pdf=pdf.distinct() #drop duplicates
# pdf.filter(pdf.item_code.isNull()).show()
pdf.na.drop().show()
pdf.count()

+-------------+----------+--------------------+--------------------+
|input_keyword| item_code|        product_name|         product_url|
+-------------+----------+--------------------+--------------------+
|      fortune|B0BSV7RDH1|glee pops date id...|https://www.amazo...|
|      fortune|B00SBMHYKS|fortune dishwashe...|https://www.amazo...|
|      fortune|B09PDZYN2X|fortune thick poh...|https://www.amazo...|
|      cadbury|B07FNXXBJB|cadbury fuse choc...|https://www.amazo...|
|      fortune|B0BS1DL5X9|fortune plus soya...|https://www.amazo...|
|      fortune|B01FDF8GB6|fortune super bas...|https://www.amazo...|
|      cadbury|B082WL4D86|cadbury dairy mil...|https://www.amazo...|
|      cadbury|B08LH4XKQF|cadbury dairy mil...|https://www.amazo...|
|      fortune|B07G3DR2FZ|fortune biryani c...|https://www.amazo...|
|      cadbury|B08C2GNX2H|cadbury dairy mil...|https://www.amazo...|
|      cadbury|B08C2QBWHT|cadbury fuse choc...|https://www.amazo...|
|      cadbury|B08Y996YQ7|cadbury 

372

In [96]:
### drop na values column-wise
for collum in pdf.columns:
    print(collum)
    pdf=pdf.filter(pdf[collum].isNotNull())
    pdf.show()
    print(pdf.count())

input_keyword
+-------------+----------+--------------------+--------------------+
|input_keyword| item_code|        product_name|         product_url|
+-------------+----------+--------------------+--------------------+
|      fortune|B0BSV7RDH1|glee pops date id...|https://www.amazo...|
|      fortune|B00SBMHYKS|fortune dishwashe...|https://www.amazo...|
|      fortune|B09PDZYN2X|fortune thick poh...|https://www.amazo...|
|      cadbury|B07FNXXBJB|cadbury fuse choc...|https://www.amazo...|
|      fortune|B0BS1DL5X9|fortune plus soya...|https://www.amazo...|
|      fortune|B01FDF8GB6|fortune super bas...|https://www.amazo...|
|      cadbury|B082WL4D86|cadbury dairy mil...|https://www.amazo...|
|      cadbury|B08LH4XKQF|cadbury dairy mil...|https://www.amazo...|
|      fortune|B07G3DR2FZ|fortune biryani c...|https://www.amazo...|
|      cadbury|B08C2GNX2H|cadbury dairy mil...|https://www.amazo...|
|      cadbury|B08C2QBWHT|cadbury fuse choc...|https://www.amazo...|
|      cadbury|B08Y9

In [97]:
pdf.collect()

[Row(input_keyword='fortune', item_code='B0BSV7RDH1', product_name='glee pops date ideas: fortune cookies box', product_url='https://www.amazon.in/Glee-Pops-Date-Ideas-Fortune/dp/B0BSV7RDH1/ref=sr_1_151?keywords=fortune&qid=1680446965&sr=8-151'),
 Row(input_keyword='fortune', item_code='B00SBMHYKS', product_name='fortune dishwasher starter combo - dishwasher detergent 1kg (pack of 1), salt 1kg (pack of 1) & rinse aid 500ml (pack of 1)', product_url='https://www.amazon.in/Fortune-Combo-Pack-Dishwasher-Starter/dp/B00SBMHYKS/ref=sr_1_12?keywords=fortune&qid=1680446946&sr=8-12'),
 Row(input_keyword='fortune', item_code='B09PDZYN2X', product_name='fortune thick poha, source of fibre and protein, 500g', product_url='https://www.amazon.in/FORTUNE-Thick-Source-Fibre-Protein/dp/B09PDZYN2X/ref=sr_1_19?keywords=fortune&qid=1680446946&sr=8-19'),
 Row(input_keyword='cadbury', item_code='B07FNXXBJB', product_name='cadbury fuse chocolate, 24 g', product_url='https://www.amazon.in/Cadbury-Fuse-Chocola

In [98]:
pdf.show()

+-------------+----------+--------------------+--------------------+
|input_keyword| item_code|        product_name|         product_url|
+-------------+----------+--------------------+--------------------+
|      fortune|B0BSV7RDH1|glee pops date id...|https://www.amazo...|
|      fortune|B00SBMHYKS|fortune dishwashe...|https://www.amazo...|
|      fortune|B09PDZYN2X|fortune thick poh...|https://www.amazo...|
|      cadbury|B07FNXXBJB|cadbury fuse choc...|https://www.amazo...|
|      fortune|B0BS1DL5X9|fortune plus soya...|https://www.amazo...|
|      fortune|B01FDF8GB6|fortune super bas...|https://www.amazo...|
|      cadbury|B082WL4D86|cadbury dairy mil...|https://www.amazo...|
|      cadbury|B08LH4XKQF|cadbury dairy mil...|https://www.amazo...|
|      fortune|B07G3DR2FZ|fortune biryani c...|https://www.amazo...|
|      cadbury|B08C2GNX2H|cadbury dairy mil...|https://www.amazo...|
|      cadbury|B08C2QBWHT|cadbury fuse choc...|https://www.amazo...|
|      cadbury|B08Y996YQ7|cadbury 

In [119]:
#collect item_code from search dataframe and pass it to get_item_details()
item_codes_list=pdf.select(f.collect_list('product_url')).first()[0] 
print(item_codes_list)
get_item_details(item_codes_list)

['https://www.amazon.in/Glee-Pops-Date-Ideas-Fortune/dp/B0BSV7RDH1/ref=sr_1_151?keywords=fortune&qid=1680446965&sr=8-151', 'https://www.amazon.in/Fortune-Combo-Pack-Dishwasher-Starter/dp/B00SBMHYKS/ref=sr_1_12?keywords=fortune&qid=1680446946&sr=8-12', 'https://www.amazon.in/FORTUNE-Thick-Source-Fibre-Protein/dp/B09PDZYN2X/ref=sr_1_19?keywords=fortune&qid=1680446946&sr=8-19', 'https://www.amazon.in/Cadbury-Fuse-Chocolate-27-5g/dp/B07FNXXBJB/ref=sr_1_63?keywords=cadbury&qid=1680446995&sr=8-63', 'https://www.amazon.in/Fortune-Plus-Soyabean-Oil-Liter/dp/B0BS1DL5X9/ref=sr_1_59?keywords=fortune&qid=1680446955&sr=8-59', 'https://www.amazon.in/Fortune-Super-Basmati-Rice-Extra/dp/B01FDF8GB6/ref=sr_1_61?keywords=fortune&qid=1680446955&sr=8-61', 'https://www.amazon.in/Cadbury-Dairy-Milk-Valentine-Chocolate/dp/B082WL4D86/ref=sr_1_92?keywords=cadbury&qid=1680446995&sr=8-92', 'https://www.amazon.in/Cadbury-Dairy-Milk-Mousse-Chocolate/dp/B08LH4XKQF/ref=sr_1_98?keywords=cadbury&qid=1680446995&sr=8-98'

C:\Users\haraa\AppData\Local\Temp\ipykernel_1792\1867251936.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:/chromedriver.exe",options=chrome_options)


|Grocery & Gourmet Foods||›||Snacks & Sweets||›||Snack Foods||›||Biscuits & Cookies||›||Butter|
{'product_name': 'Glee Pops Date Ideas: Fortune Cookies Box', 'code': 'https://www.amazon.in/Glee-Pops-Date-Ideas-Fortune/dp/B0BSV7RDH1/ref=sr_1_151?keywords=fortune&qid=1680446965&sr=8-151', 'brand': 'Glee Pops', 'mrp': '₹999', 'final_price': '₹599', 'category': '|Grocery & Gourmet Foods||›||Snacks & Sweets||›||Snack Foods||›||Biscuits & Cookies||›||Butter|', 'rating': '5 out of 5', 'offers': '-40%', 'produuct_status': 'In stock', 'image_count': 6, 'prod_url': 'https://www.amazon.in/Glee-Pops-Date-Ideas-Fortune/dp/B0BSV7RDH1/ref=sr_1_151?keywords=fortune&qid=1680446965&sr=8-151', 'video_count': ''}
|Health & Personal Care||›||Household Supplies||›||Dishwashing Supplies||›||Dishwasher Detergents||›||Liquid Detergent|
{'product_name': '', 'code': 'https://www.amazon.in/Fortune-Combo-Pack-Dishwasher-Starter/dp/B00SBMHYKS/ref=sr_1_12?keywords=fortune&qid=1680446946&sr=8-12', 'brand': '', 'mrp':

['B0BSV7RDH1',
 'B00SBMHYKS',
 'B09PDZYN2X',
 'B07FNXXBJB',
 'B0BS1DL5X9',
 'B01FDF8GB6',
 'B082WL4D86',
 'B08LH4XKQF',
 'B07G3DR2FZ',
 'B08C2GNX2H',
 'B08C2QBWHT',
 'B08Y996YQ7',
 'B08SKGBH86',
 'B01A4BI0J2',
 'B0BT716WBZ',
 'B07KKF5YDL',
 'B0764MLJJW',
 'B0735HS26S',
 'B07F478V1J',
 'B0BJVRJV7M',
 'B07H4C231S',
 'B077G1XWK7',
 'B0B9YN7MM3',
 'B00LIVC89G',
 'B06ZY11F14',
 'B08FC229BZ',
 'B000MOEUMS',
 'B0777L97WQ',
 'B08217GY8L',
 'B01IHCPDJA',
 'B00W5CFUY2',
 'B071R2KKMZ',
 'B06XPFVRSL',
 'B07KKC6G32',
 'B07KKDRHCZ',
 'B01N3B4HGP',
 'B07KKDVG2V',
 'B0916NWTN2',
 'B0B7GPX6W8',
 'B081S2JHZK',
 'B095W1554Z',
 'B075757RF2',
 'B00SL32GVY',
 'B07FR6RLP2',
 'B019Z85HXC',
 'B07P87ZZYS',
 'B0916QR3QW',
 'B077HV6CX6',
 'B0C11F23Q4',
 'B09Z6ZC6PC',
 'B077TKK4RN',
 'B07HGQ5VPJ',
 'B07KKFYRLW',
 'B0BNBG3QDJ',
 'B0738KH7XD',
 'B0757631XR',
 'B08C2JLQ9N',
 'B07KKDM6VY',
 'B013UV2K58',
 'B09GG1QK7D',
 'B07F46MDKY',
 'B08GXSJJQV',
 'B07H6JRFGR',
 'B06XH9QPX2',
 'B08218JCLQ',
 'B092QS7PXV',
 'B01AA3J8

In [ ]:
df.load()

In [ ]:
search_items()